# Codeado por Franco M. Di Maria

# Guia 2 : Spark

## Funciones auxiliares 

In [175]:
import random
import time

# By Tom Alsberg and Boris (StackOverflow) 
def str_time_prop(start, end, format, prop):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formated in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(format, time.localtime(ptime))


def random_date(start, end):
    return str_time_prop(start, end, '%Y-%m-%d', random.random())

## Ejercicio 1

1-Se tiene un RDD con el registro de notas de los alumnos de la forma (padrón, materia, nota, fecha). Se pide resolver utilizando PySpark:
* A.Cuántos alumnos aprobaron al menos 1 materia en los últimos 2 años.
* B. Un RDD conteniendo el promedio de notas de cada alumno de la forma (padrón, promedio).
* C.El nombre y apellido del alumno con mejor promedio. Para esto pued eutilizarse un segundo RDD alumnos con registros (padron, nombre y apellido).

### Set de datos

In [102]:
pre_data_1_notas = [
    (100498, 'Datos', 10, '2019-9-16'),
    (100499, 'Datos', 9, '2019-9-17'),
    (100500, 'Datos', 8, '2019-9-18'),
    (100501, 'Datos', 7, '2019-9-19'),
    (100502, 'Datos', 7, '2019-9-20'),
    (100498, 'Taller', 9, '2019-6-1'),
    (100499, 'Taller', 6, '2019-6-2'),
    (100500, 'Taller', 8, '2019-6-3'),
    (100501, 'Taller', 8, '2019-6-4'),
    (100502, 'Taller', 9, '2019-6-5'),
    (100498, 'Proba', 8, '2018-11-11'),
    (100499, 'Proba', 6, '2018-11-12'),
    (100500, 'Proba', 6, '2018-11-13'),
    (100501, 'Proba', 7, '2018-11-14'),   
    (100502, 'Proba', 8, '2018-11-15'),
]

rdd_1 = sc.parallelize(pre_data_1_notas)
#rdd_1.collect()

### A.Cuántos alumnos aprobaron al menos 1 materia en los últimos 2 años.

##### RESOLUCION 1

In [144]:
rdd_1.filter( lambda x: (x[2] >= 4) & (x[3] > "2018-1-1") & (x[3] < "2018-12-31") )\
    .map( lambda x: x[0] )\
    .distinct()\
    .count()

5

##### RESOLUCION 2

In [143]:
rdd_1.filter( lambda x: (x[2] >= 4) & (x[3] > "2018-1-1") & (x[3] < "2018-12-31") )\
    .flatMap( lambda x: (x[0],) )\
    .distinct()\
    .count()

5

##### RESOLUCION 3

In [142]:
rdd_1.filter( lambda x: (x[2] >= 4) & (x[3] > "2018-1-1") & (x[3] < "2018-12-31") )\
    .map( lambda x: (x[0],1) )\
    .reduceByKey( lambda x, y: x )\
    .count()

5

### B. Un RDD conteniendo el promedio de notas de cada alumno de la forma (padrón, promedio).

In [150]:
rdd_1_promedios = rdd_1.map( lambda x: (x[0], (x[2], 1)) )\
                    .reduceByKey( lambda x, y: (x[0] + y[0], x[1] + y[1]) )\
                    .map( lambda x: (x[0], x[1][0] / x[1][1]) )
rdd_1_promedios.collect()

[(100498, 9.0),
 (100499, 7.0),
 (100500, 7.333333333333333),
 (100501, 7.333333333333333),
 (100502, 8.0)]

### C.El nombre y apellido del alumno con mejor promedio. Para esto puede utilizarse un segundo RDD alumnos con registros (padron, nombre y apellido).

#### Set de datos auxiliar

In [151]:
pre_data_1_alumnos = [
    (100498, 'Franco Martin', 'Di Maria'),
    (100499, 'Alumno B', 'Ble Ble'),
    (100500, 'Alumno C', 'Bli Bli'),
    (100501, 'Alumno D', 'Blo Blo'),
    (100502, 'Alumno E', 'Blu Blu'),
]

rdd_1_alumnos = sc. parallelize(pre_data_1_aux)

##### RESOLUCION 1 

In [159]:
rdd_1_alumnos_por_padron = rdd_1_alumnos.map( lambda x: (x[0], x[1] + ' ' + x[2]) )
padron_promedio_max = rdd_1_promedios.reduce(lambda x, y: x if x[1] > y[1] else y)
rdd_1_alumnos_por_padron.filter(lambda x: x[0] == padron_promedio_max[0])\
    .map(lambda x: (x[0], x[1], padron_promedio_max[1]) )\
    .take(1)

[(100498, 'Franco Martin Di Maria', 9.0)]

##### RESOLUCION 2

In [164]:
rdd_1_alumnos_por_padron = rdd_1_alumnos.map( lambda x: (x[0], x[1] + ' ' + x[2]) )
rdd_1_alumnos_por_padron.join(rdd_1_promedios)\
    .map( lambda x: (x[0], x[1][0], x[1][1]) )\
    .reduce( lambda x, y: x if x[2] > y[2] else y )\

(100498, 'Franco Martin Di Maria', 9.0)

## Ejercicio 2

2- Se tiene un RDD registros de ventas de producto con la forma (fecha de venta, código de
producto, precio de venta) y en otro RDD detalle de los productos con (código de producto,
descripción del producto, categoría). Se pide resolver utilizando PySpark:
* A. Cuál es el producto más vendido.
* B. Cuál es la categoría de productos más vendida.
* C. Cuál es el top5 de productos más vendidos generando un RDD con (código de producto, descripción, cantidad de ventas)
* D. Cuál es el producto que registró mayor aumento de precio en el último año, tomando para este análisis solo los productos que cuenten con al menos 50 ventas en el último año.
* E. Idem anterior, pero calculando la categoría de productos que registró mayor variación de precios en el último año.

### Sets de datos 

In [208]:
pre_data_2_ventas = []
pre_data_2_detalles = []
codigo_productos = [i for i in range(10)]
for _ in range (50):
    pre_data_2_ventas.append( 
        ( random_date('2018-1-1', '2019-12-31'), random.choice(codigo_productos), random.uniform(100, 1000) ) 
    )
    pre_data_2_detalles.append( 
        ( random.choice(codigo_productos), 'Descripcion ...' , random.choice(['Categ. A', 'Categ. B', 'Categ. C']) ) 
    )
rdd_2_ventas = sc.parallelize(pre_data_2_ventas)
rdd_2_detalles = sc.parallelize(pre_data_2_detalles)

### A. Cuál es el producto más vendido. 

In [216]:
rdd_2_ventas.map( lambda x: (x[1], 1) )\
    .reduceByKey( lambda x, y: x + y )\
    .reduce(lambda x, y: x if x[1] > y[1] else y)[0]

6

### B. Cuál es la categoría de productos más vendida.